### Generate training data

use pure text and label

In [9]:
def combine_data(path):
    rating = [1, 2, 3, 4, 5]
    with open('pure_text_data.tsv', 'w') as f:
        for r in rating:
            rfile = path + 'Office_Products_5.json.gz_' + str(r) + '.0.txt'
            if r <= 3:
                label = 0
            else:
                label = 1
            with open(rfile, 'r') as rf:
                for line in rf:
                    f.write(line.strip('\n') + '\t' + str(label) + '\n')

In [10]:
path = 'data/'
combine_data(path)

In [11]:
total_data = []
with open('pure_text_data.tsv', 'r') as f:
    for line in f:
        sent, label = line.strip('\n').split('\t')
        total_data.append((sent, label))

In [13]:
import random
random.shuffle(total_data)

In [14]:
train_data = total_data[:int(len(total_data)*0.7)]
dev_data = total_data[int(len(total_data)*0.7):int(len(total_data)*0.85)]
test_data = total_data[int(len(total_data)*0.85):]

In [15]:
with open('train_data.tsv', 'w') as f:
    for d in train_data:
        f.write(d[0] + '\t' + d[1] + '\n')
with open('dev_data.tsv', 'w') as f:
    for d in dev_data:
        f.write(d[0] + '\t' + d[1] + '\n')
with open('test_data.tsv', 'w') as f:
    for d in test_data:
        f.write(d[0] + '\t' + d[1] + '\n')

### fit logistic regression model

In [25]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = []
labels = []
with open('train_data.tsv', 'r') as f:
    for line in f:
        sent, label = line.strip('\n').split('\t')
        corpus.append(sent)
        labels.append(int(label))
        
test_corpus = []
test_labels = []
with open('test_data.tsv', 'r') as f:
    for line in f:
        sent, label = line.strip('\n').split('\t')
        test_corpus.append(sent)
        test_labels.append(int(label))

In [26]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus+test_corpus)

In [27]:
print(type(X))
print(X.shape)
print(len(labels))

<class 'scipy.sparse.csr.csr_matrix'>
(610907, 83864)
503099


In [28]:
train_X = X[:len(corpus)]
test_X = X[len(corpus):]

In [30]:
print(train_X.shape)
print(len(labels))
print(test_X.shape)
print(len(test_labels))

(503099, 83864)
503099
(107808, 83864)
107808


In [35]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(class_weight = 'balanced').fit(train_X, labels)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [36]:
pred = clf.predict(test_X)

In [38]:
correct = 0
for y, p in zip(test_labels, list(pred)):
    if y == p:
        correct += 1
acc = correct/len(test_labels)
print(acc)

0.8832925200356189


In [39]:
pos = 0
neg = 0
for y in test_labels:
    if y == 1:
        pos += 1
    else:
        neg += 1
print(pos)
print(neg)

94008
13800
